<a href="https://colab.research.google.com/github/Abelbrown/h-m-kaggle-project/blob/main/model_als_iteration1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Objective of iteration 1: manually code an ALS algorithm (even though the Implicit library provides one)

In [ ]:
!pip install --upgrade implicit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#Quick summary of the Alternating Leastr Squares ('ALS')


The 'ALS' (Alternating Least Squares) algorithm aims to decompose a user-item matrix into a scalar product of two matrices of lower dimensions, where one describes the properties of users and the other describes the properties of items for sale (principle of "matrix factorization"). This approximation of the original matrix into two "sub-matrices" is done in such a way that a cost function (composed of the squared error and a regularization term) is minimized. Each matrix will share "latent factors" represented as columns for one matrix and rows for the other. These latent factors represent projections of explanatory variables into a lower-dimensional space. ALS follows the same principle as linear regression: the algorithm establishes an iterative process consisting of two steps. In each step, the model fixes one matrix and finds the features of the other matrix while ensuring that the cost function decreases or at worst remains stable (because OLS, from which ALS is derived, guarantees a minimum squared error).

The model will learn to factorize our initial matrix into two representations (a user matrix and an item matrix). This will enable it to accurately predict the next items that a customer is likely to purchase. In this case, we need to predict 12 H&M items that a user is likely to buy: these 12 items will be those with embedded vectors close to the customer's embedded vectors.

##Imports 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os; os.environ['OPENBLAS_NUM_THREADS']='1'
import numpy as np
import pandas as pd
import implicit
from scipy.sparse import coo_matrix
from implicit.evaluation import mean_average_precision_at_k

##Loading files

In [ ]:
df_cust = pd.read_csv('/content/drive/MyDrive/customers.csv')
df_cust_sample = df_cust.sample(frac=0.05)
df_art = pd.read_csv('/content/drive/MyDrive/articles.csv', dtype={'article_id': str})
df_art_sample = df_art.sample(frac=0.05)
df_trans = pd.read_csv('/content/drive/MyDrive/transactions_train.csv', dtype={'article_id': str}, parse_dates=['t_dat'])
df_trans_sample = df_trans.sample(frac=0.05)
df_sub = pd.read_csv('/content/drive/MyDrive/sample_submission.csv')

##Préparation de la donnée en vue de la création de deux matrices (U et V)

In [ ]:
#Option 1: Select only the transaction rows where the date is greater than August 21, 2020 to minimize processing time.
df_trans_bis = df_trans[df_trans['t_dat'] > '2020-08-21']
df_trans_bis.head()

#Option 2: Select 5% of the original file as a sample -> df_trans_sample (see cell above)
##df_trans_sample.head()

,t_dat,customer_id,article_id,price,sales_channel_id
30597413,2020-08-22,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,0913688003,0.033881,2
30597414,2020-08-22,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,0913688003,0.033881,2
30597415,2020-08-22,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,0923460001,0.042356,2
30597416,2020-08-22,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,0934380001,0.050831,2
30597417,2020-08-22,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,0913688001,0.033881,2


In [ ]:
#Creation of lists of users id and articles id
list_cust_id = df_cust['customer_id'].unique().tolist()
list_art_id = df_art['article_id'].unique().tolist()

In [ ]:
#Create a dictionary with the list of user ids and the list of article ids.
#Get the (index, value) pairs with enumerate, then transform into a list and then into a dictionary.
cust_ids = dict(list(enumerate(list_cust_id))) 
art_ids = dict(list(enumerate(list_art_id)))
#Display the first few lines.
print(list(cust_ids.items())[:6]) 
'\n' 
print(list(art_ids.items())[:6])

[(0, '00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657'), (1, '0000423b00ade91418cceaf3b26c6af3dd342b51fd051eec9c12fb36984420fa'), (2, '000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318'), (3, '00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2c5feb1ca5dff07c43e'), (4, '00006413d8573cd20ed7128e53b7b13819fe5cfc2d801fe7fc0f26dd8d65a85a'), (5, '000064249685c11552da43ef22a5030f35a147f723d5b02ddd9fd22452b1f5a6')]
[(0, '0108775015'), (1, '0108775044'), (2, '0108775051'), (3, '0110065001'), (4, '0110065002'), (5, '0110065011')]


In [ ]:
#Assign customer and article indices to the transaction table
## Invert the key-value pairs of our dictionary created in the previous cell. 
cust_map = {u: uidx for uidx, u in cust_ids.items()} 
## Substitute each customer_id with the created index and assign them to a new column 'cust_id'
df_trans_bis['cust_id'] = df_trans_bis['customer_id'].map(cust_map) 

art_map = {a: aidx for aidx, a in art_ids.items()} 
df_trans_bis['art_id'] = df_trans_bis['article_id'].map(art_map) 
df_trans_bis.head(10)

<ipython-input-13-e90d9c97466f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trans_bis['cust_id'] = df_trans_bis['customer_id'].map(cust_map)
<ipython-input-13-e90d9c97466f>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trans_bis['art_id'] = df_trans_bis['article_id'].map(art_map)


,t_dat,customer_id,article_id,price,sales_channel_id,cust_id,art_id
30597413,2020-08-22,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,0913688003,0.033881,2,38,103595
30597414,2020-08-22,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,0913688003,0.033881,2,38,103595
30597415,2020-08-22,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,0923460001,0.042356,2,38,104483
30597416,2020-08-22,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,0934380001,0.050831,2,38,105214
30597417,2020-08-22,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,0913688001,0.033881,2,38,103593
30597418,2020-08-22,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,0572998001,0.042356,2,38,17125
30597419,2020-08-22,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,0866387011,0.042356,2,38,95404
30597420,2020-08-22,00159bfc26c2cf788c09789f006fa698b24d0f1dbd8309...,0751471001,0.030492,1,467,67522
30597421,2020-08-22,00159bfc26c2cf788c09789f006fa698b24d0f1dbd8309...,0927922002,0.030492,1,467,104840
30597422,2020-08-22,002562bc0fe3f01f08e14874067a22907b02c98c701077...,0579541086,0.015237,1,785,18613


## Initialization step of two matrices U and V 

In [ ]:
#Constructing the two matrices U and V from the data of the main matrix (row, col, data)
##We will create tuples (cust_id, art_id) and count how many times they appear. For example, if the
##tuple (38, 103595) appears twice, we will enter the value 2 in our main matrix (at the intersection of these two numbers). In reality, we
##will only construct matrices U and V in order to establish our cost function later on.

###Step 1: Insert the respective indices of customers and articles into row and col (of the main matrix)
row = df_trans_bis['cust_id'].values
col = df_trans_bis['art_id'].values

###Step 2: Creating tuples and counting occurrences
from collections import Counter
tuples = [(row[i], col[i]) for i in range(len(row))]
matrix_m_values = dict(Counter(tuples))

row = np.array(list(matrix_m_values.keys()))[:,0] #First element of each key to obtain unique values
col = np.array(list(matrix_m_values.keys()))[:,1]

###Step 3: Assign the values of each tuple to data (of the main matrix)
data = matrix_m_values.values()

###Step 4: Creating matrices U and V with random values

#-->Option1: Random numbers following a normal distribution (drawbacks: float numbers and negative values)
##Matrix U of size (m, k) with m as the number of unique customers and k as a hyperparameter (latent factors) that will be randomly set

m = len(df_cust['customer_id'].unique())
k = 100
U = np.random.randn(m, k).astype('float64')

##Matrix V of size (k, n) with n as the number of unique articles and k as the same hyperparameter

n = len(df_art['article_id'].unique())
k = 100
V = np.random.randn(k, n).astype('float64')

#-->Option2: Random integers
#==> first: Obtain the value of the largest tuple in the 'tuples' dictionary to indicate it in the step of creating matrices U and V
#max_key = max(matrix_m_values, key=matrix_m_values.get) # get the key with the maximum value
#max_val = matrix_m_values[max_key] # get the value associated with that key
#==> Second: Initialization of matrices U and V
#m = len(df_cust['customer_id'].unique())
#n = len(df_art['article_id'].unique())
#k = 100
#U = np.random.randint(0, max_val+1, size=(m, k)) # Adding +1 because the upper limit max_val is exclusive (numpy random.randint documentation)
#V = np.random.randint(0, max_val+1, size=(k, n)) # Adding +1 because the upper limit max_val is exclusive (numpy random.randint documentation)
#U = U.astype(np.float64)
#V = V.astype(np.float64)


In [ ]:
from pprint import pprint
pprint(U)

pprint(V)

array([[-0.61048449,  1.7565281 ,  0.56040945, ..., -0.58960806,
         1.58523003, -0.94967993],
       [ 1.46591441,  0.33229104,  0.79532913, ...,  0.05563889,
         0.48928827, -0.99114805],
       [-0.28818692,  1.32041446,  0.42943286, ...,  1.36956256,
         0.36545452,  0.60042544],
       ...,
       [ 0.63748512, -0.25351444, -0.02123696, ...,  0.11151226,
        -0.73640665,  0.10989766],
       [ 0.30064314,  0.12623435,  1.15654082, ...,  0.00178675,
         0.44039888,  0.18120908],
       [-0.25943057, -0.15067444,  0.96574323, ...,  0.57642845,
        -1.23754377,  1.00287739]])
array([[ 0.53096593,  0.92404781, -1.20990108, ...,  0.40048765,
         0.75923716,  0.91007106],
       [ 1.12840632,  0.23704459, -0.08023114, ...,  0.90207145,
         0.32927342,  0.04660526],
       [-0.6431553 ,  0.01634481, -0.2200867 , ..., -0.59734973,
        -0.22114237,  0.75722974],
       ...,
       [ 1.58964778,  1.14181414,  0.26936195, ..., -0.1553823 ,
         1

# Loss function and Gradient Descent

In [ ]:
from time import time
t0 = time()
eps = 0.1
for i in range(len(row)):
  """
  L = (data[i] - U[row[i],:]@V[:,col[i])**2 #general form of the cost function that allows looping only over the known values of the main matrix
  """
  L_prime_U = -2*V[:,col[i]]*(list(data)[i] - U[row[i],:]@V[:,col[i]]) # derivative with respect to U
  L_prime_V = -2*U[row[i],:]*(list(data)[i] - U[row[i],:]@V[:,col[i]]) # derivative with respect to V
  U[row[i],:] -= eps*L_prime_U # Gradient descent with a step represented by eps. We add each new value to the U matrix
  V[:,col[i]] -= eps*L_prime_V # Same for V

t1 = time()
print("Execution time of the algorithm:", t1-t0)


<ipython-input-16-d52fb533fb8c>:9: RuntimeWarning: overflow encountered in multiply
  L_prime_V = -2*U[row[i],:]*(list(data)[i] - U[row[i],:]@V[:,col[i]]) # derivative with respect to V
<ipython-input-16-d52fb533fb8c>:8: RuntimeWarning: invalid value encountered in matmul
  L_prime_U = -2*V[:,col[i]]*(list(data)[i] - U[row[i],:]@V[:,col[i]]) # derivative with respect to U
<ipython-input-16-d52fb533fb8c>:9: RuntimeWarning: invalid value encountered in matmul
  L_prime_V = -2*U[row[i],:]*(list(data)[i] - U[row[i],:]@V[:,col[i]]) # derivative with respect to V
<ipython-input-16-d52fb533fb8c>:8: RuntimeWarning: overflow encountered in multiply
  L_prime_U = -2*V[:,col[i]]*(list(data)[i] - U[row[i],:]@V[:,col[i]]) # derivative with respect to U
<ipython-input-16-d52fb533fb8c>:8: RuntimeWarning: overflow encountered in matmul
  L_prime_U = -2*V[:,col[i]]*(list(data)[i] - U[row[i],:]@V[:,col[i]]) # derivative with respect to U
<ipython-input-16-d52fb533fb8c>:9: RuntimeWarning: overflow encoun

## Testing cells 

In [ ]:
#def f(x):
  #return (x-1)**2+(1/x-2)**2

#def df(x):
  #return 2*(x-1) + 2*(1/x-2)*(-1/x**2)

#def descente1(f, df, a, alpha=1e-2, eps=1e-9, maxIter=1000):
  # Finds the minimum of a function f using gradient descent
  # df should be the derivative of f
  # a is the initial value
  # alpha is the learning rate that determines the descent speed (default: 1/100)
  # eps is the desired precision (default: 1/1000000000)
  # maxIter is the maximum number of iterations (default: 1000)

  #grad = df(a)
  #i=0
  #while abs(grad)>eps: # as long as the slope is not approximately zero
      #grad = df(a) # calculate the slope
      #a = a-alpha*grad # take a small step downwards
      #i += 1
      #print(i, a, grad) # uncomment this line to print the iterations
      #if i > maxIter:
          #return None # abandon if the number of iterations is too high
  #return a

#def descente2(f, a, lr=1e-2, eps=1e-9, maxIter=1000):
  # Finds the minimum of a function f using gradient descent with numerical derivative
  # a is the initial value
  # alpha is the learning rate that determines the descent speed (default: 1/100)
  # eps is the desired precision (default: 1/1000000000)
  # maxIter is the maximum number of iterations (default: 1000)
  #grad = 1
  #i=0
  #while abs(grad)>eps:
    #grad = (f(a+eps)-f(a-eps))/(2*eps) # numerical approximation of the derivative
    #a = a-lr*grad
    #i += 1
    #print(i, a, grad) # uncomment this line to print the iterations
    #if i > maxIter:
        #return None
  #return a



In [ ]:
#Another example
#import numpy as np

#def gradient_descent(start, gradient, learn_rate, max_iter, tol=0.01):
  #steps = [start] # history tracking
  #x = start

  #for _ in range(max_iter):
    #diff = learn_rate*gradient(x)
    #if np.abs(diff)<tol:
    #break
    #x = x - diff
    #steps.append(x) # history tracing

  #return steps, x